# Database Integration Test
Test PostgreSQL connection and persistence

In [ ]:
import psycopg2
import os
from datetime import datetime
import json

In [ ]:
# Database connection
def get_db_connection():
    return psycopg2.connect(
        host="localhost",
        database="react_agent_db",
        user="agent_user",
        password="agent_pass",
        port="5432"
    )

# Test connection
try:
    conn = get_db_connection()
    cursor = conn.cursor()
    cursor.execute("SELECT version();")
    version = cursor.fetchone()
    print(f"✅ Database connected: {version[0]}")
    cursor.close()
    conn.close()
except Exception as e:
    print(f"❌ Database connection error: {e}")

In [ ]:
# Test conversation storage
def save_conversation(session_id, user_message, agent_response, tool_calls=None):
    conn = get_db_connection()
    cursor = conn.cursor()
    
    cursor.execute("""
        INSERT INTO conversations (session_id, user_message, agent_response, tool_calls)
        VALUES (%s, %s, %s, %s)
        RETURNING id
    """, (session_id, user_message, agent_response, json.dumps(tool_calls) if tool_calls else None))
    
    conversation_id = cursor.fetchone()[0]
    conn.commit()
    cursor.close()
    conn.close()
    return conversation_id

# Test save
conv_id = save_conversation(
    "test-session",
    "Calculate integral of x^2 from 0 to 2",
    "The integral is 8/3",
    {"tool": "calculate_integral", "params": {"function": "x**2", "x_min": 0, "x_max": 2}}
)
print(f"✅ Conversation saved with ID: {conv_id}")

In [ ]:
# Test agent state storage
def save_agent_state(session_id, state_data):
    conn = get_db_connection()
    cursor = conn.cursor()
    
    cursor.execute("""
        INSERT INTO agent_state (session_id, state_data) 
        VALUES (%s, %s)
        ON CONFLICT (session_id) 
        DO UPDATE SET state_data = %s, updated_at = CURRENT_TIMESTAMP
    """, (session_id, json.dumps(state_data), json.dumps(state_data)))
    
    conn.commit()
    cursor.close()
    conn.close()

# Test state save
save_agent_state("test-session", {"last_calculation": "x^2 integral", "result": 8/3})
print("✅ Agent state saved")